In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

print(device)
block_size = 64
batch_size = 128
max_iters = 3000
learning_rate = 3e-4
eval_iters = 100
n_embd = 384
n_head = 8
n_layer = 8
dropout = 0.2

mps


In [2]:
chars = ""
with open('Gita.txt','r', encoding='utf-8') as f:
    text = f.read()
    chars = sorted(list(set(text)))
    
print(chars)
vocublary_size = len(chars)
print(vocublary_size)



['\n', ' ', '!', '"', '#', '&', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '[', ']', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
79


In [3]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}

encode = lambda s:[string_to_int[c] for c in s]
decode = lambda l:''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text),dtype=torch.long)
print(data[:100])

tensor([ 0,  0, 28, 57, 56, 61, 55, 53, 72, 61, 67, 66,  0,  0, 44, 39,  1, 33,
        38, 28, 33, 25,  0,  0, 43, 67,  1, 60, 53, 74, 57,  1, 33,  1, 70, 57,
        53, 56,  1, 72, 60, 61, 71,  1, 75, 67, 66, 56, 57, 70, 58, 73, 64,  1,
        53, 66, 56,  1, 71, 68, 61, 70, 61, 72, 10, 72, 60, 70, 61, 64, 64, 61,
        66, 59,  1, 71, 68, 57, 57, 55, 60,  9,  0, 26, 77,  1, 35, 70, 61, 71,
        60, 66, 53,  1, 53, 66, 56,  1, 40, 70])


In [4]:
n = int(0.8*len(data))

train_data = data[:n]
val_data = data[n:]
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[56,  1, 72,  ...,  1, 57, 70],
        [56, 71,  0,  ..., 57, 71, 71],
        [75, 60, 57,  ...,  1,  1, 26],
        ...,
        [73, 55, 60,  ..., 67, 75, 57],
        [ 0,  1,  1,  ...,  1, 72, 60],
        [53, 72, 57,  ...,  0,  1,  1]], device='mps:0')
targets:
tensor([[ 1, 72, 57,  ..., 57, 70, 70],
        [71,  0,  1,  ..., 71, 71,  1],
        [60, 57, 66,  ...,  1, 26, 73],
        ...,
        [55, 60, 57,  ..., 75, 57, 72],
        [ 1,  1, 37,  ..., 72, 60, 67],
        [72, 57, 56,  ...,  1,  1, 29]], device='mps:0')


In [5]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print('when input is', context, 'target is', target)

when input is tensor([0]) target is tensor(0)
when input is tensor([0, 0]) target is tensor(28)
when input is tensor([ 0,  0, 28]) target is tensor(57)
when input is tensor([ 0,  0, 28, 57]) target is tensor(56)
when input is tensor([ 0,  0, 28, 57, 56]) target is tensor(61)
when input is tensor([ 0,  0, 28, 57, 56, 61]) target is tensor(55)
when input is tensor([ 0,  0, 28, 57, 56, 61, 55]) target is tensor(53)
when input is tensor([ 0,  0, 28, 57, 56, 61, 55, 53]) target is tensor(72)


In [6]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [7]:
from torch import Block

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   
        q = self.query(x) 
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) 
        wei = F.softmax(wei, dim=-1) 
        wei = self.dropout(wei)
        v = self.value(x) 
        out = wei @ v 
        return out


class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1) # (B, T, F) -> (B, T, [h1, h1, h1, h1, h2, h2, h2, h2, h3, h3, h3, h3])
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    
     def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
     def forward(self, x):
        y = self.sa(x)
        x = self.ln1(x + y)
        y = self.ffwd(x)
        x = self.ln2(x + y)
        return x


class GPTLanguageModel(nn.Module):
    def __init__(self, vocublary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocublary_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head = n_head) for _ in range(n_layer)])

        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocublary_size)

        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def forward(self, index, targets=None):
        B, T = index.shape

        tok_emb = self.token_embedding_table(index) 
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) 
        x = tok_emb + pos_emb
        x = self.blocks(x) 
        x = self.ln_f(x) 
        logits = self.lm_head(x)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :] 
            probs = F.softmax(logits, dim=-1) 
            index_next = torch.multinomial(probs, num_samples=1) 
            index = torch.cat((index, index_next), dim=1) 
        return index

model = GPTLanguageModel(vocublary_size)
m = model.to(device)


#context = torch.zeros((1,1), dtype=torch.long, device=device)
#generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
#print(generated_chars)

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.430, val loss: 4.426
step: 100, train loss: 2.275, val loss: 2.303
step: 200, train loss: 2.094, val loss: 2.125
step: 300, train loss: 1.986, val loss: 2.039
step: 400, train loss: 1.934, val loss: 1.995
step: 500, train loss: 1.883, val loss: 1.949
